Welcome to Quantopian! The Getting Started Tutorial serves as an introduction to writing and backtesting an algorithm on Quantopian. This tutorial covers many of the basics of the Quantopian API, and is designed for those who are new to Quantopian. This tutorial is divided into a series of lessons, with each one focusing on a different part of the API. By the end, we will work up to a simple mean reversion algorithm. The video series also covers the same material.
What is a Trading Algorithm?
On Quantopian, a trading algorithm is a Python program that defines two special functions: initialize() and handle_data(). initialize() is called when the program is started, and handle_data() is called once per minute during simulation or live-trading in events that we'll refer to as 'bars'. The job of initialize() is to perform any one-time startup logic. The job of handle_data() is to decide what orders, if any, should be placed each minute.
The following is an example of an algorithm that allocates 100% of its portfolio in AAPL:

In [22]:
def initialize(context):
    # Reference to AAPL
    context.aapl = sid(24)

def handle_data(context, data):
    # Position 100% of our portfolio to be long in AAPL
    order_target_percent(context.aapl, 1.00)

To run this example algorithm, create a copy by clicking the "Clone" button in the upper right hand corner. Run a backtest by clicking "Build Algorithm" (top left) or "Run Full Algorithm" (top right).
Datasets
When writing an algorithm on Quantopian, you have access to free minute bar historical pricing and volume data for US equities (covered in this tutorial) as well as free Morningstar fundamentals data, and third-party datasets such as news sentiment, and earnings calendars. Futures data is coming soon but is not yet available. For more information on futures, click here.
Continue on to lesson 2 to begin writing your first algorithm.

### Core Functions
An algorithm on Quantopian has 3 core functions: *initialize()*, *handle_data()*, and *before_trading_start()*. The first one, initialize(), must be implemented in every algorithm while handle_data() and before_trading_start() are optional. These functions each have a different role:
initialize()
initialize() is called exactly once when our algorithm starts and requires context as input.
context is an augmented Python dictionary used for maintaining state during our backtest or live trading, and can be referenced in different parts of our algorithm. context should be used instead of global variables in the algorithm. Properties can be accessed using dot notation (context.some_property).
handle_data()
handle_data() is called once at the end of each minute and requires context and data as input. context is a reference to the same dictionary in initialize() and data is an object that stores several API functions which we will discuss in a later lesson.
The following example stores a string variable with the value 'hello' in context, and prints it out each minute.

In [23]:
def initialize(context):
    context.message = 'hello'


    
def before_trading_start(context, data): 
   
 pass
    
    
def handle_data(context, data):
    print(context.message)    

*before_trading_start()*
before_trading_start() is called once per day before the market opens and requires context and data as input. It is frequently used when selecting securities to order. This will be explored in a later tutorial.



#### Referencing Securities
The best algorithms select securities to trade dynamically using pipeline. Pipeline is an advanced tool and will be covered in a later tutorial. To keep things simple, we'll start by manually referencing hand-picked securities. There are two ways to manually select securities on Quantopian: the *sid() function*, and the *symbol() function*.
sid()
The sid() function returns a security given a unique integer security ID (SID). sid() is a robust way to reference a security. The SID of a security remains constant despite ticker changes and as such, is a reliable way to reference a stock. A security's SID never changes.
For example, if we want to reference AAPL, we can use sid(), which brings up a text prompt allowing us to fill in a ticker symbol and gives us the corresponding security ID.


_symbol()_
Alternatively, the symbol() function returns a security given a ticker symbol. symbol() is not robust to ticker changes unless you specify a reference date with set_symbol_lookup_date('YYYY-MM-DD'). It is strongly discouraged to us symbol() to reference a security without setting a reference date.

In [24]:
def initialize(context):
    context.aapl = sid(24)

def handle_data(context, data):
    print(context.aapl)


#### Ordering Securities
There are several functions that can be used to order securities in the Quantopian API. In this tutorial, we are going to focus on the *order_target_percent() function*. *order_target_percent()* allows us to order securities to a target percent of our portfolio value (sum of open positions value + cash balance).
order_target_percent() requires two arguments: the asset being ordered, and the target percent of the portfolio. For example, the following line would order 50% of our portfolio value worth of AAPL:

*order_target_percent()* requires two arguments: the asset being ordered, and the target percent of the portfolio. For example, the following line would order 50% of our portfolio value worth of AAPL:

In [25]:
#order_target_percent(sid(24), 0.50)

To open a short position (a position that profits when the security will drop in price), we can simply enter a negative target percent:

In [26]:
#order_target_percent(sid(24), -0.50)

The following example takes a long position in Apple stock worth 60% of our portfolio value, and takes a short position in the SPY ETF worth 40% of our portfolio value:

In [27]:
def initialize(context):
    context.aapl = sid(24)
    context.spy = sid(8554)

def handle_data(context, data):
    # Note: data.can_trade() is explained in the next lesson
    if data.can_trade(context.aapl):
        order_target_percent(context.aapl, 0.60)
    if data.can_trade(context.spy):
        order_target_percent(context.spy, -0.40)

#### The data Object
The *object* contains functions that allow us to look up current or historical pricing and volume data for any security. data is available in *handle_data()* and *before_trading_start()*, as well as any scheduled functions. Let's have a look at some of data's functions:
*data.current()*
*data.current()* can be used to retrieve the most recent value of a given field(s) for a given asset(s). data.current() requires two arguments: the asset or list of assets, and the field or list of fields being queried. Possible fields include 'price', 'open', 'high', 'low', 'close', and 'volume'. The output type will depend on the input types. To get the most recent price of AAPL, we can use:

In [29]:
#data.current(sid(24), 'price')

The following line of code gets the most recent price of AAPL and SPY and returns a pandas Series indexed by asset:

In [30]:
#data.current([sid(24), sid(8554)], 'price')

The following line will get the last known low and high prices of AAPL and SPY from and returns a pandas DataFrame (indexed by assets, fields as columns):

In [31]:
#data.current([sid(24), sid(8554)], ['low', 'high'])

*data.can_trade()*
data.can_trade() is used to determine if an asset(s) is currently listed on a supported exchange and can be ordered. If data.can_trade() returns True for a particular asset in a given minute bar, we are able to place an order for that asset in that minute. This is an important guard to have in our algorithm if we hand-pick the securities that we want to trade. It requires a single argument: an asset or a list of assets. The following example checks if AAPL is currently listed on a major exchange:

In [32]:
#data.can_trade(sid(24))

data.history()
data.history() is covered in the next lesson.


##### The history() Function
The data object has a function  <font color='blue'>history()</font>  that allows us to get trailing windows of historical pricing or volume data.<font color='red'>data.history()</font>  requires 4 arguments: an asset or list of assets, a field or list of fields, an integer lookback window length, and a lookback frequency. Possible fields include 'price', 'open', 'high', 'low', 'close', and 'volume'. Possible frequencies are '1d' for daily and '1m' for minutely. The following example returns a pandas Series containing the price history of AAPL over the last 10 days and uses <font color='red'>pandas.Series.mean()</font> to calculate the mean.


In [2]:
 # Get the 10-day trailing price history of AAPL in the form of a Series.
#hist = data.history(sid(24), 'price', 10, '1d')

# Mean price over the last 10 days.
#mean_price = hist.mean()

Note: With '1d' frequency, the most recent value in the result from data.history() will include a value for the current date in the simulation, which can sometimes be a value for a partial day. For example, if data.history() is called in the first minute of the day, the last row of the returned DataFrame will represent values from 9:31AM, whereas the previous 9 rows will represent end-of-day values. To get the past 10 complete days of data, we can get an extra day of data, and drop the most recent row. The following example gets the trading volume of SPY from the last 10 complete days:

In [3]:
#data.history(sid(8554), 'volume', 11, '1d')[:-1].mean()

Similar to the other data functions, the return type of data.history() depends on the input types. In the next example, the return type is a pandas DataFrame indexed by date, with assets as columns:

In [5]:
# Get the last 5 minutes of volume data for each security in our list.
#hist = data.history([sid(24), sid(8554), sid(5061)], 'volume', 5, '1m')

# Calculate the mean volume for each security in our DataFrame.
#mean_volumes = hist.mean(axis=0)

And if we pass a list of fields, we get a pandas Panel indexed by field, having date as the major axis, and assets as the minor axis:

In [6]:
# Low and high minute bar history for each of our securities.
#hist = data.history([sid(24), sid(8554), sid(5061)], ['low', 'high'], 5, '1m')

# Calculate the mean low and high over the last 5 minutes
#means = hist.mean()
#mean_lows = means['low']
#mean_highs = means['high']

The following example is clonable and can be backtested in the IDE. It prints the mean trading volume of a list of securities over the last 10 minutes:

In [8]:
def initialize(context):
    # AAPL, MSFT, SPY
    context.security_list = [sid(24), sid(8554), sid(5061)]

def handle_data(context, data):
    hist = data.history(context.security_list, 'volume', 10, '1m').mean()
    print(hist.mean())


##### Trading Calendars
Before running an algorithm, we need to select the trading calendar on which we want our algorithm to run. Currently, there are two options: the US Equities Calendar and the US Futures Calendar.
The trading calendar can be selected from a dropdown menu before running a backtest.
Help calendar dropdown
If you are only trading equities, you will want to use the US Equities Calendar. If you would like to include futures in your algorithm, you will need to use the US Futures Calendar. For the rest of this tutorial, we will use the US Equities Calendar, which runs from 9:30AM-4PM Eastern time on regular trading days and follows the NYSE holiday and early close schedule.

##### Scheduling Functions
So far we have seen that trade logic can exist in  <font color='red'>handle_data()</font> which runs every minute. But what if we want to trade at a different frequency like once per day, or once per month? schedule_function() allows us to schedule custom functions at regular intervals and allows us to specify interday frequency as well as intraday timing.
The following example schedules a function called <font color='red'>rebalance()</font>to run once per day, one hour after market open (usually 10:30AM ET).


schedule_function(func=rebalance,
                  date_rules=date_rules.every_day(),
                  time_rules=time_rules.market_open(hours=1))

<font color='red'>rebalance()</font> must be a function requiring context and data as arguments that we can define later in our algorithm.
Note: The relative date and time rules that we specify will follow the trading calendar that we selected. For example, <font color='red'>date_rules.market_open()</font> will usually run in the 9:30AM ET minute bar if we selected the US equity calendar.
In another example, let's say we want to run a custom function <font color='red'>weekly_trades()</font>, on the last trading day of each week, 30 minutes before market close. We can use:

In [ ]:
m=[1,2,3,4,5]

print(m)

f=(31.69304+m*(-0.3352))*0.955945

print(f)